In [1]:
pwd

'C:\\Users\\erfan\\Downloads\\plugin\\plugin\\smart-contract-vulnerability-detection\\test_contracts'

In [2]:
import json, os, re

In [8]:
a = [{
        "vulnerableLines": "1-6",
        "vulnerabilityReason": "The function does not properly protect itself from reentrancy exploits. Additionally, it does not properly check for integer underflow on redeemableEther_39[msg.sender] balance.",
        "potentialSecurityRisk": "Reentrancy attacks can enable an attacker to drain the contract of its ether. While underflow can turn a negative balance into a large positive balance due to uint properties.",
        "fixedCode": "mapping(address => uint) redeemableEther_39; bool private _reentrancyLock = false; modifier reentrancyGuard() { require(!_reentrancyLock, 'Reentrant call'); _reentrancyLock = true; _; _reentrancyLock = false; } function claimReward_39() public reentrancyGuard { require(redeemableEther_39[msg.sender] > 0); uint transferValue_39 = redeemableEther_39[msg.sender]; redeemableEther_39[msg.sender] = 0; msg.sender.transfer(transferValue_39); }"
    }
]

In [33]:
predict = a[0]['vulnerabilityReason']
print(predict)

The function does not properly protect itself from reentrancy exploits. Additionally, it does not properly check for integer underflow on redeemableEther_39[msg.sender] balance.


In [34]:
with open ("tested/label_12.json", 'r', encoding = 'utf-8') as file:
    label = json.load(file)

In [35]:
print(label['vulnerabilityReason'])

The function transfers funds to the user before resetting the redeemable balance, making it susceptible to reentrancy.


In [38]:
!pip install -q sentence-transformers

ERROR: Could not install packages due to an OSError: [WinError 145] The directory is not empty: 'c:\\users\\erfan\\onedrive - polymtlus\\apps\\lib\\site-packages\\sympy\\logic\\tests'

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\erfan\\OneDrive - polymtlus\\Apps\\Lib\\site-packages\\sympy-1.14.0.dist-info\\INSTALLERk3kxt9cm.tmp'



In [7]:
with open("tested/contract_13_12.txt", 'r', encoding = 'utf-8') as file:
    content = file.read()
    
pattern = r' Integer Overflow/Underflow:\s*(\[\s*{.*?}\s*\])'
matches = re.findall(pattern, content, flags=re.DOTALL)

all_answers = []
for match in matches:
    try:
        json_block = json.loads(match)
        all_answers.append(json_block)
    except json.JSONDecodeError as e:
        print("Failed to parse one block:", e)


with open('tested/contract_13_12.json', 'w', encoding='utf-8') as outfile:
    for block in all_answers:
        json.dump(block, outfile)
        outfile.write('\n')  # Write each JSON object on a new line


print("Saved", len(all_answers), "answer blocks to tested/contract_13_12.json")

Saved 0 answer blocks to tested/contract_13_12.json


In [18]:
a = []
with open("../test_contracts/results.json", "r", encoding="utf-8") as f:
    for line in f:
        a.append(json.loads(line))


In [28]:
print(a[1][1])

{'vulnerableLines': '13-19', 'vulnerabilityReason': 'The function is vulnerable to re-entrancy due to a call to an external contract before updating the state. The external call is potentially dangerous as it could be influenced by the timestamp.', 'potentialSecurityRisk': "An attacker could manipulate the timestamp to re-enter the function and potentially claim the bonus multiple times, essentially draining the contract's funds.", 'fixedCode': 'function getFirstWithdrawalBonus(address recipient) public {\nrequire(!claimedBonus[recipient]);\nrewardsForA[recipient] += 100;\nclaimedBonus[recipient] = true;\nwithdrawReward(recipient);\n}'}


In [ ]:
from sentence_transformers import SentenceTransformer, util
import json
        
# Load lightweight sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Safe JSON parser
def safe_parse_json(field):
    if not field or field.strip() == "":
        return {}
    if isinstance(field, dict):
        return field
    try:
        return json.loads(field)
    except json.JSONDecodeError:
        return {}

# Compute semantic similarity between label and predicted vulnerabilityReason
factual_scores = []

for sample in a:  # Use all samples
    label = safe_parse_json(sample.get("label", ""))
    pred = safe_parse_json(sample.get("predict", ""))

    label_reason = label.get("vulnerabilityReason", "")
    pred_reason = pred.get("vulnerabilityReason", "")

    if not label_reason.strip() or not pred_reason.strip():
        continue

    label_emb = model.encode(label_reason, convert_to_tensor=True)
    pred_emb = model.encode(pred_reason, convert_to_tensor=True)

    score = util.cos_sim(label_emb, pred_emb).item()
    factual_scores.append(score)

# Report
avg_score = sum(factual_scores) / len(factual_scores) if factual_scores else 0
print(f"Average Factual Consistency Score (Label ↔ Predict Reason): {avg_score:.4f}")


## extract sol from json and save into .sol:

In [ ]:
import json
import os
import re

# Input file path (change as needed)
input_file = "dataset.json"

def extract_solidity_code(text):
    # Remove leading number and colon (e.g., '1: ')
    lines = text.split('\n')
    clean_lines = []
    for line in lines:
        # Remove the line number prefix like "23: "
        match = re.match(r'^\d+:\s*(.*)', line)
        if match:
            clean_lines.append(match.group(1))
    return '\n'.join(clean_lines)

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)  # assumes this is a list of samples

for idx, sample in enumerate(data):
    raw_input = sample.get("input", "")
    solidity_code = extract_solidity_code(raw_input)
    
    # Save to .sol file
    file_path = os.path.join( f"contract_{idx+1}.sol")
    with open(file_path, "w", encoding="utf-8") as sol_file:
        sol_file.write(solidity_code)
      # Parse and save output field as label_i.json
    output_raw = sample.get("output", "")
    try:
        output_data = json.loads(output_raw)
        label_filename = f"label_{idx}.json"
        with open(label_filename, "w", encoding="utf-8") as label_file:
            json.dump(output_data, label_file, indent=2)
    except json.JSONDecodeError as e:
        print(f"Failed to parse output for sample {idx}: {e}")

    print(f" Saved contract_{idx}.sol and label_{idx}.json")
